## 7.3 高度な設定のためのコールバック関数

In [1]:
import json
import pandas as pd
import numpy as np
from typing import Union
from sklearn.datasets import load_diabetes

from plotly import graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Template
from plotly import callbacks

# DiabetesデータセットのDataFrameを読み込み
df_X, df_y = load_diabetes(return_X_y=True, as_frame=True, scaled=False)
df = pd.concat([df_X, df_y], axis=1)

df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,59.0,2.0,32.1,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0
1,48.0,1.0,21.6,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0
2,72.0,2.0,30.5,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0
3,24.0,1.0,25.3,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0
4,50.0,1.0,23.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,2.0,28.2,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0
438,47.0,2.0,24.9,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0
439,60.0,2.0,24.9,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0
440,36.0,1.0,30.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0


In [2]:
# 独自テンプレートを読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Traceを作成
trace = go.Scatter(x=df['age'], y=df['target'], mode='markers')

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Diabetes dataset',
    xaxis={'title': 'age'},
    yaxis={'title': 'disease progression'}
)

# FigureWidgetを作成
widget = go.FigureWidget(trace, layout)

widget

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '5e4fb52f-c6c2-403d-8b9b-44a1733b7d65',
              'x': array([59., 48., 72., ..., 60., 36., 36.]),
              'y': array([151.,  75., 141., ..., 132., 220.,  57.])}],
    'layout': {'template': '...',
               'title': {'text': 'Diabetes dataset'},
               'xaxis': {'title': {'text': 'age'}},
               'yaxis': {'title': {'text': 'disease progression'}}}
})

In [3]:
default_size = 6
default_color = '#1F77B4'
changed_color = '#ff7f0e'

def click_funcion(trace:go.Trace, points:callbacks.Points, selector:callbacks.InputDeviceState)->None:
    """マーカーがクリックされた際のコールバック関数

    Args:
        trace (go.Trace): 対象のTrace
        points (callbacks.Points): プロットされたポイント
        selector (callbacks.InputDeviceState): セレクター
    """
    N = len(trace.x)    # データ個数
    
    if selector.ctrl == True:
        # CTRLキーありでクリックされた場合
        size = [default_size] * N
        color = [default_color] * N
    else:
        if hasattr(trace.marker.size, '__iter__') == False:
            # marker.sizeがtupleではない場合listを作成
            size = [trace.marker.size] * N
            color = [trace.marker.color] * N
        else:
            size = list(trace.marker.size)  # tupleからlistに変換
            color = list(trace.marker.color)
        
        point_index = points.point_inds[0]  # クリックされたインデックス

        size[point_index] = default_size * 3    # マーカーサイズを3倍に変更
        color[point_index] = changed_color      # マーカー色を変更
    
    # マーカーの更新
    with widget.batch_update():
        trace.marker.color = color
        trace.marker.size = size

In [4]:
# コールバック関数を設定
widget.data[0].on_click(click_funcion)

widget

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '5e4fb52f-c6c2-403d-8b9b-44a1733b7d65',
              'x': array([59., 48., 72., ..., 60., 36., 36.]),
              'y': array([151.,  75., 141., ..., 132., 220.,  57.])}],
    'layout': {'template': '...',
               'title': {'text': 'Diabetes dataset'},
               'xaxis': {'title': {'text': 'age'}},
               'yaxis': {'title': {'text': 'disease progression'}}}
})

In [5]:
# FigureWidgetを作成
widget = go.FigureWidget(trace, layout)

def select_funcion(trace:go.Trace, points:callbacks.Points, selector:Union[callbacks.BoxSelector, callbacks.LassoSelector]) -> None:
    """マーカーが範囲選択された際のコールバック関数

    Args:
        trace (go.Trace): 対象のTrace
        points (callbacks.Points): 選択されたポイント
        selector (Union[callbacks.BoxSelector, callbacks.LassoSelector]): セレクタ−
    """
    # アノテーションの作成
    annotation = go.layout.Annotation(
        x=np.min(points.xs),    # x位置は選択範囲の最小箇所
        y=np.max(points.ys),    # y位置は選択範囲の最大箇所
        text=f'selection mean<br>x:{np.mean(points.xs):.2f}<br>y:{np.mean(points.ys):.2f}',
        showarrow=True          # 選択範囲とアノテーションをつなぐ線を表示
    )

    # FigureWidgetのLayoutを更新
    widget.update_layout(annotations=[annotation])

def deselect_function(trace:go.Trace, points:callbacks.Points)->None:
    """選択解除された際のコールバック関数

    Args:
        trace (go.Trace): 対象のTrace
        points (callbacks.Points): ポイント
    """
    # Annotationの作成
    annotation = go.layout.Annotation(text='', showarrow=False)

    widget.update_layout(annotations=[annotation])

# 選択時の動作を設定
widget.data[0].on_selection(select_funcion)

# 選択解除の動作を設定
widget.data[0].on_deselect(deselect_function)

widget

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '2ed05d35-347b-4e35-b753-0ed8d05799b1',
              'x': array([59., 48., 72., ..., 60., 36., 36.]),
              'y': array([151.,  75., 141., ..., 132., 220.,  57.])}],
    'layout': {'template': '...',
               'title': {'text': 'Diabetes dataset'},
               'xaxis': {'title': {'text': 'age'}},
               'yaxis': {'title': {'text': 'disease progression'}}}
})

In [6]:
# 1行2列のFigureWidgetを作成
widget = go.FigureWidget(make_subplots(rows=1, cols=2))

# 1行1列目にTraceを追加
widget.add_trace(
    go.Scatter(x=df['age'], y=df['target'], mode='markers', name='disease progression'),
    row=1,
    col=1
)

# 1行2列目にTraceを追加
widget.add_trace(
    go.Scatter(x=df['bmi'], y=df['s2'], mode='markers', name='low-density lipoproteins'),
    row=1,
    col=2
)
widget.add_trace(
    go.Scatter(x=df['bmi'], y=df['s3'], mode='markers', name='high-density lipoproteins'),
    row=1,
    col=2
)

# Layoutを作成
layout = go.Layout(
    template=template,
    title='Diabetes dataset',
    xaxis={'title': 'age'},
    yaxis={'title': 'disease progression'},
    xaxis2={'title': 'BMI'},
    yaxis2={'title': 'lipoproteins'}
)

widget.update_layout(layout)

widget

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'disease progression',
              'type': 'scatter',
              'uid': 'f81840c4-f546-4f73-8b9d-e6ef61ed1931',
              'x': array([59., 48., 72., ..., 60., 36., 36.]),
              'xaxis': 'x',
              'y': array([151.,  75., 141., ..., 132., 220.,  57.]),
              'yaxis': 'y'},
             {'mode': 'markers',
              'name': 'low-density lipoproteins',
              'type': 'scatter',
              'uid': '353d58cf-c2b8-47c9-a0fc-1334930d2fab',
              'x': array([32.1, 21.6, 30.5, ..., 24.9, 30. , 19.6]),
              'xaxis': 'x2',
              'y': array([ 93.2, 103.2,  93.6, ..., 106.6, 125.2, 133.2]),
              'yaxis': 'y2'},
             {'mode': 'markers',
              'name': 'high-density lipoproteins',
              'type': 'scatter',
              'uid': '2eda39e5-0e45-49c0-af17-d71d2db10460',
              'x': array([32.1, 21.6, 30.5, ..., 24.9, 30. , 19.6]

In [7]:
default_size = 6    # 標準のマーカーサイズ

def update_sizes(sizes:np.ndarray)->None:
    """マーカーサイズを更新

    Args:
        sizes (np.ndarray): マーカーサイズの配列
    """
    traces = widget.data    # Traceのlistを取得
    with widget.batch_update():
        for trace, size in zip(traces, sizes):
            trace.marker.size = size    # 各Traceのマーカーサイズを更新

def hover_function(trace:go.Trace, points:callbacks.Points, selector:callbacks.InputDeviceState)->None:
    """マーカーがフォーカスされた際のコールバック関数

    Args:
        trace (go.Trace): 対象のTrace
        points (callbacks.Points): フォーカスされたポイント
        selector (callbacks.InputDeviceState): セレクター
    """
    N = len(trace.x)    # マーカー個数
    marker_sizes = np.full([3, N], fill_value=default_size)     # 3つのTraceすべてのマーカーサイズ
    
    if points.point_inds != []:
        index = points.point_inds[0]    # 1番めのTraceでフォーカスされたマーカーのインデックス
        marker_sizes[:, index] *= 3     # フォーカスされたマーカーを各Traceでサイズ3倍
    
    update_sizes(marker_sizes)

def unhover_function(trace:go.Trace, points:callbacks.Points, selector:callbacks.InputDeviceState)->None:
    """フォーカス解除された際のコールバック関数

    Args:
        trace (go.Trace): 対象のTrace
        points (callbacks.Points): フォーカス解除されたポイント
        selector (callbacks.InputDeviceState): セレクター
    """
    N = len(trace.x)    # マーカー個数
    marker_sizes = np.full([3, N], fill_value=default_size)     # 標準のサイズで配列作成
    
    update_sizes(marker_sizes)

In [8]:
widget.data[0].on_hover(hover_function)
widget.data[0].on_unhover(unhover_function)

widget

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'disease progression',
              'type': 'scatter',
              'uid': 'f81840c4-f546-4f73-8b9d-e6ef61ed1931',
              'x': array([59., 48., 72., ..., 60., 36., 36.]),
              'xaxis': 'x',
              'y': array([151.,  75., 141., ..., 132., 220.,  57.]),
              'yaxis': 'y'},
             {'mode': 'markers',
              'name': 'low-density lipoproteins',
              'type': 'scatter',
              'uid': '353d58cf-c2b8-47c9-a0fc-1334930d2fab',
              'x': array([32.1, 21.6, 30.5, ..., 24.9, 30. , 19.6]),
              'xaxis': 'x2',
              'y': array([ 93.2, 103.2,  93.6, ..., 106.6, 125.2, 133.2]),
              'yaxis': 'y2'},
             {'mode': 'markers',
              'name': 'high-density lipoproteins',
              'type': 'scatter',
              'uid': '2eda39e5-0e45-49c0-af17-d71d2db10460',
              'x': array([32.1, 21.6, 30.5, ..., 24.9, 30. , 19.6]